# Trying AWS Console with BOTO3 
How to create EC2 Instance using Boto3 

# Create Key-pair for secure connection

In [1]:
import boto3

In [2]:
ec2 = boto3.client("ec2")
ec2.describe_instances()

{'Reservations': [],
 'ResponseMetadata': {'RequestId': '3ed463a3-0662-4d76-a3d4-f3e699e09f9a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3ed463a3-0662-4d76-a3d4-f3e699e09f9a',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '219',
   'date': 'Fri, 29 Aug 2025 08:12:06 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [3]:
# Creating Key pair 
resp  = ec2.create_key_pair(KeyName="testuser")

## Create a pem file 

In [4]:
key=resp["KeyMaterial"]
file = open("./creds/testuser.pem", "w+")
file.write(key)
file.close()

# Create an Amazon EC2 Instance 

In [8]:
ec2.describe_instances()

response = ec2.run_instances(
    ImageId = 'ami-0c4fc5dcabc9df21d',
    MinCount = 1,
    MaxCount = 1,
    InstanceType = 't3.micro',
    KeyName = 'testuser',
    BlockDeviceMappings = [
        {
            "DeviceName": "/dev/xvda",
            "Ebs" : {
                "DeleteOnTermination": True,
                "VolumeSize": 20
            }
        }
    ]
)

In [13]:
# Check the created instance
ec2.describe_instances()

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 0,
     'ImageId': 'ami-0c4fc5dcabc9df21d',
     'InstanceId': 'i-087e56f348d2a023e',
     'InstanceType': 't3.micro',
     'KeyName': 'testuser',
     'LaunchTime': datetime.datetime(2025, 8, 29, 8, 16, tzinfo=tzutc()),
     'Monitoring': {'State': 'disabled'},
     'Placement': {'AvailabilityZone': 'eu-north-1a',
      'GroupName': '',
      'Tenancy': 'default'},
     'PrivateDnsName': 'ip-172-31-29-0.eu-north-1.compute.internal',
     'PrivateIpAddress': '172.31.29.0',
     'ProductCodes': [],
     'PublicDnsName': 'ec2-13-51-178-236.eu-north-1.compute.amazonaws.com',
     'PublicIpAddress': '13.51.178.236',
     'State': {'Code': 16, 'Name': 'running'},
     'StateTransitionReason': '',
     'SubnetId': 'subnet-064cec0bf36cd98b4',
     'VpcId': 'vpc-0f835a8a4d6fa6501',
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
       'Ebs': {'AttachTime': datetime.datetime(2025, 8, 29

# Create a Security group and add it to EC2 Instance 

In [17]:
response = ec2.describe_security_groups()
response

{'SecurityGroups': [{'Description': 'Security group for testing',
   'GroupName': 'testuser',
   'IpPermissions': [],
   'OwnerId': '508426718808',
   'GroupId': 'sg-0e943ad903051d7fb',
   'IpPermissionsEgress': [{'IpProtocol': '-1',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'UserIdGroupPairs': []}],
   'VpcId': 'vpc-0f835a8a4d6fa6501'},
  {'Description': 'default VPC security group',
   'GroupName': 'default',
   'IpPermissions': [{'IpProtocol': '-1',
     'IpRanges': [],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'UserIdGroupPairs': [{'GroupId': 'sg-0981e486cc9b77ac3',
       'UserId': '508426718808'}]}],
   'OwnerId': '508426718808',
   'GroupId': 'sg-0981e486cc9b77ac3',
   'IpPermissionsEgress': [{'IpProtocol': '-1',
     'IpRanges': [{'CidrIp': '0.0.0.0/0'}],
     'Ipv6Ranges': [],
     'PrefixListIds': [],
     'UserIdGroupPairs': []}],
   'VpcId': 'vpc-0f835a8a4d6fa6501'}],
 'ResponseMetadata': {'RequestId': 'bfe

In [15]:
security_response = ec2.create_security_group(
    GroupName = 'testuser',
    Description = "Security group for testing"
)

In [18]:
security_response

{'GroupId': 'sg-0e943ad903051d7fb',
 'ResponseMetadata': {'RequestId': '893ee29c-d528-4553-ab6d-56a1390ab80b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '893ee29c-d528-4553-ab6d-56a1390ab80b',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '374',
   'date': 'Fri, 29 Aug 2025 09:31:25 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [22]:
# Extract group id to apply inbound rules....
security_group_id  = security_response['GroupId']
security_group_id

'sg-0e943ad903051d7fb'

## Provide Inbound Rules

In [23]:
# ip, port, traffic type
response = ec2.authorize_security_group_ingress(
    GroupId = security_group_id,
    IpPermissions = [
        {
            'IpProtocol': 'tcp',
            'FromPort': 22,
            'ToPort': 22,
            'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
        }
    ]
)

In [24]:
response

{'Return': True,
 'SecurityGroupRules': [{'SecurityGroupRuleId': 'sgr-066e5916d0e368bbc',
   'GroupId': 'sg-0e943ad903051d7fb',
   'GroupOwnerId': '508426718808',
   'IsEgress': False,
   'IpProtocol': 'tcp',
   'FromPort': 22,
   'ToPort': 22,
   'CidrIpv4': '0.0.0.0/0'}],
 'ResponseMetadata': {'RequestId': '64c06dc4-f17b-40f0-8642-64f207127092',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '64c06dc4-f17b-40f0-8642-64f207127092',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '697',
   'date': 'Fri, 29 Aug 2025 09:51:41 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

## Connect the security group with EC2 Instance 

In [27]:
response = ec2.describe_instances()
response

{'Reservations': [{'Groups': [],
   'Instances': [{'AmiLaunchIndex': 0,
     'ImageId': 'ami-0c4fc5dcabc9df21d',
     'InstanceId': 'i-087e56f348d2a023e',
     'InstanceType': 't3.micro',
     'KeyName': 'testuser',
     'LaunchTime': datetime.datetime(2025, 8, 29, 8, 16, tzinfo=tzutc()),
     'Monitoring': {'State': 'disabled'},
     'Placement': {'AvailabilityZone': 'eu-north-1a',
      'GroupName': '',
      'Tenancy': 'default'},
     'PrivateDnsName': 'ip-172-31-29-0.eu-north-1.compute.internal',
     'PrivateIpAddress': '172.31.29.0',
     'ProductCodes': [],
     'PublicDnsName': 'ec2-13-51-178-236.eu-north-1.compute.amazonaws.com',
     'PublicIpAddress': '13.51.178.236',
     'State': {'Code': 16, 'Name': 'running'},
     'StateTransitionReason': '',
     'SubnetId': 'subnet-064cec0bf36cd98b4',
     'VpcId': 'vpc-0f835a8a4d6fa6501',
     'Architecture': 'x86_64',
     'BlockDeviceMappings': [{'DeviceName': '/dev/xvda',
       'Ebs': {'AttachTime': datetime.datetime(2025, 8, 29

In [34]:
# Get instance id 
instance_id = response['Reservations'][0]['Instances'][0]['InstanceId']
instance_id, security_group_id

('i-087e56f348d2a023e', 'sg-0e943ad903051d7fb')

In [43]:
# Get the group old id
old_gid = response['Reservations'][0]['Instances'][0]['SecurityGroups'][0]['GroupId']
old_gid

'sg-0981e486cc9b77ac3'

In [44]:
# Connect the security group with instance id 
# in groups list add the security groups - multiple groups is allowed
response = ec2.modify_instance_attribute(
    InstanceId = instance_id,
    Groups = [security_group_id]
)
response

{'ResponseMetadata': {'RequestId': '48a252f7-f1ca-4f01-b8a0-394ef0ec22ca',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '48a252f7-f1ca-4f01-b8a0-394ef0ec22ca',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'content-type': 'text/xml;charset=UTF-8',
   'content-length': '235',
   'date': 'Fri, 29 Aug 2025 11:18:54 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

# Start, Stop, and Delete Amazon EC2 instances 

In [56]:
# Status -> running, stopped, terminated, pending, etc.........
import time
def wait_for_status(instance_id, target_status):
    while True:
        response = ec2.describe_instances(InstanceIds = [instance_id])
        state = response['Reservations'][0]['Instances'][0]['State']['Name']

        if state == target_status:
            print(f"Instance {instance_id} is in {state} state")
            break
        time.sleep(10)


def start_instance(instance_id):
    print(f"Starting EC2 Instance {instance_id}")
    ec2.start_instances(InstanceIds = [instance_id])
    wait_for_status(instance_id, 'running')


def stop_instance(instance_id):
    print(f"stopping EC2 Instance {instance_id} ")
    ec2.stop_instances(InstanceIds = [instance_id])
    wait_for_status(instance_id, 'stopped')


def terminate_instance(instance_id):
    print(f"Terminating EC2 Instance {instance_id}")
    ec2.terminate_instances(InstanceIds = [instance_id])
    wait_for_status(instance_id, 'terminated')
# stop_instance(instance_id)
# start_instance(instance_id)

In [57]:
response = ec2.describe_instances(InstanceIds = [instance_id])
response['Reservations'][0]['Instances'][0]['State']['Name']

'running'

In [58]:
terminate_instance(instance_id)

Terminating EC2 Instance i-087e56f348d2a023e
Instance i-087e56f348d2a023e is in terminated state


# Thank you 